# Bootes Brown photo-z cross

The photometric redshifts are calculated for the Brown aperture matched catalogue. Since in the manufacture of the masterlist we take the Brown aperture matched catalogue and add in some other obejcts. We here asign the Brown objects with their help id

In [1]:
from astropy.table import Table, join, Column
import numpy as np

In [2]:
photo_z = Table.read('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public.fits')[
    'ID',
    'ID_OPTICAL', 
    'z1_median'
]
cross_ids = Table.read('../../dmu1/dmu1_ml_Bootes/data/'
                     + 'master_list_cross_ident_bootes_20190701.fits')[
    'help_id', 
    'brown_i_id'
]
cross_ids['brown_i_id'].name = 'ID'


photo_z['ID'].fill_value = -99
photo_z = photo_z.filled()

FileNotFoundError: [Errno 2] No such file or directory: './data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public.fits'

In [3]:
new_cat = join(photo_z, cross_ids, join_type='left')

In [6]:
new_cat[new_cat['z1_median']<0]['z1_median'] = np.nan

In [7]:
new_cat[:10].show_in_notebook()

idx,ID,ID_OPTICAL,z1_median,help_id
0,-99,1000000,4.645,--
1,0,1,3.6887,--
2,1,2,0.2606,--
3,2,3,4.9513,--
4,3,4,4.4549,--
5,4,5,4.4835,--
6,5,6,2.7475,--
7,6,7,4.4733,--
8,7,8,0.1774,--
9,8,9,0.1334,--


In [8]:
new_cat.write('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public_with_help_id.fits', overwrite=True)